In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
import json
from tqdm import tnrange
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
with open('/content/gdrive/My Drive/dataset/Line/aozora_authors.json', 'r') as myfile:
    data = myfile.read()
authors = json.loads(data)

with open('/content/gdrive/My Drive/dataset/Line/aozora_titles.json', 'r') as myfile:
    data = myfile.read()
titles = json.loads(data)

with open('/content/gdrive/My Drive/dataset/Line/aozora_contents.json', 'r') as myfile:
    data = myfile.read()
contents = json.loads(data)

with open('/content/gdrive/My Drive/dataset/Line/train_aozora_data.json', 'r') as myfile:
    data = myfile.read()
train_data = json.loads(data)

with open('/content/gdrive/My Drive/dataset/Line/train_q.json', 'r') as myfile:
    data = myfile.read()
train_q = json.loads(data)

with open('/content/gdrive/My Drive/dataset/Line/test_aozora_data.json', 'r') as myfile:
    data = myfile.read()
test_data = json.loads(data)

with open('/content/gdrive/My Drive/dataset/Line/test_q.json', 'r') as myfile:
    data = myfile.read()
test_q = json.loads(data)

with open('/content/gdrive/My Drive/dataset/Line/suggestion.json.sample', 'r') as myfile:
    data = myfile.read()
suggestion = json.loads(data)

In [ ]:
print(len(authors))

In [ ]:
print((train_data[1]['novels'][0]))

In [ ]:
print(contents[train_data[1]['novels'][0]['content_id']])

In [ ]:
author_list = []
title_list = []
content_list = []
for i in range(len(train_data)):
  for j in range(len(train_data[i]['novels'])):
    if train_data[i]['author_id'] == 'auth_0610':
      print(contents[train_data[i]['novels'][j]['content_id']])
    content_item = contents[train_data[i]['novels'][j]['content_id']]
    content_item = content_item.replace('\n', '')
    content_item = content_item.replace('\r', '')
    if len(content_item) != 0:
      author_list.append(train_data[i]['author_id'])
      title_list.append(titles[train_data[i]['novels'][j]['title_id']])
      content_list.append(content_item)

train_df = pd.DataFrame(list(zip(author_list, title_list, content_list)), columns=["author", "title", "content"])

In [ ]:
print('auth_0113' in train_df['author'].values)

In [ ]:
print((train_df))

In [ ]:
author_list = []
content_list = []
for i in tnrange(len(train_df)):
  for j in range(0, len(train_df["content"][i]), 256):
    author_list.append(train_df['author'][i])
    content_list.append(train_df['content'][i][j:j+256])
decompose_256 = pd.DataFrame(list(zip(author_list, content_list)), columns=["author", "content"])

In [ ]:
print(len(decompose_256))

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(decompose_256['author'].values)
y = le.transform(decompose_256['author'].values)

In [ ]:
print(y)
print(le.inverse_transform(y))

In [ ]:
print('auth_0610' in le.inverse_transform(y))

In [ ]:
y = pd.get_dummies(y).values

In [ ]:
print(y)

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(decompose_256['content'].values)

In [ ]:
print(len(tokenizer.word_counts))

In [ ]:
from keras.preprocessing.sequence import pad_sequences
X = tokenizer.texts_to_sequences(decompose_256['content'].values)
X = pad_sequences(X, maxlen=256)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout, BatchNormalization,TimeDistributed
from keras.callbacks import EarlyStopping

drop_rate = 0.1
model = Sequential()
model.add(Embedding(len(tokenizer.word_counts), 128, input_length=X_train.shape[1]))
model.add(BatchNormalization())
model.add(SpatialDropout1D(drop_rate))
model.add(LSTM(256, dropout=drop_rate, recurrent_dropout=drop_rate))
model.add(BatchNormalization())
model.add(Dropout(drop_rate))
model.add(Dense(256, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
files.download('model.png')

In [ ]:
print(model.summary())

In [ ]:
epochs = 14
batch_size = 256
es = EarlyStopping(monitor='val_loss', patience=5, mode='auto', restore_best_weights=True)
for i in range(4):
  history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data = [X_val, y_val], callbacks=[es], verbose=1)
  model.save('line_coding_test_model_epoch_' + str(epochs) +'.h5')
  files.download('line_coding_test_model_epoch_' + str(epochs) +'.h5')
  epochs += 5

In [ ]:
model.save('line_coding_test_model_epoch_15.h5')


In [ ]:
files.download('line_coding_test_model_epoch_15.h5')

In [ ]:
from keras.models import load_model
model = load_model('/content/gdrive/My Drive/dataset/Line/line_coding_test_model_epoch_15.h5')

In [ ]:
author_list = []
title_list = []
content_list = []
for i in range(len(test_q)):
    author_list.append("unknown author")
    title_list.append(test_q[i]['title_id'])
    content_item = contents[test_data[test_q[i]['title_id']]]
    content_item = content_item.replace('\n', '')
    content_item = content_item.replace('\r', '')
    content_list.append(content_item)

test_df = pd.DataFrame(list(zip(author_list, title_list, content_list)), columns=["author", "title", "content"])

In [ ]:
print(test_df.head(5))

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
test_pred_result_rank = []
for i in tnrange(len(test_df)):
#for i in tqdm(range(2)):
    #pred = [0] * y_train.shape[1]
    if len(test_df["content"][i]) == 0:
        for j in range(len(test_q[i]['candidates'])):
            candidates.append(test_q[i]['candidates'][j])
        test_pred_result_rank.append(candidates)
    else:
        X_test = []
        for j in range(0, len(test_df["content"][i]), 64):
            X_test.append(test_df["content"][i][j:j+256])
        X_test = tokenizer.texts_to_sequences(X_test)
        X_test = pad_sequences(X_test, maxlen=256)
        pred = model.predict(X_test)
        pred = np.sum(pred, axis=0)

        sorted_index = np.flip(np.argsort(pred),0)
        sorted_author = []
        sorted_author_conf = []
        sorted_q_author = []
        sorted_q_author_conf = []
        candidates = []
        for j in range(len(sorted_index)):
            sorted_author.append(le.inverse_transform([sorted_index[j]])[0])
            sorted_author_conf.append(pred[sorted_index[j]])
        for j in range(len(test_q[i]['candidates'])):
            candidates.append(test_q[i]['candidates'][j])
        for j in range(len(sorted_author)):
            if sorted_author[j] in candidates:
                sorted_q_author.append(sorted_author[j])
                sorted_q_author_conf.append(sorted_author_conf[j])
        test_pred_result_rank.append(sorted_q_author)

In [ ]:
with open('/content/gdrive/My Drive/dataset/Line/test_q.json', 'r') as myfile:
    data = myfile.read()
test_q = json.loads(data)

In [ ]:
print(set(test_q[1]['candidates']))
print(set(test_pred_result_rank[1]))
print(set(test_q[i]['candidates'])-set(test_pred_result_rank[i]))
print(list(set(test_q[i]['candidates'])-set(test_pred_result_rank[i]))[0])
test_q[1]['candidates'].append(list(set(test_q[i]['candidates'])-set(test_pred_result_rank[i]))[0])
print(test_q[1]['candidates'])

In [ ]:
type_a = 0
type_c = 0
for i in range(len(test_q)):
  if (set(test_q[i]['candidates']) != set(test_pred_result_rank[i])):
    if (len(test_q[i]['candidates']) > len(test_pred_result_rank[i])):
      type_a += 1
      missing_element = list(set(test_q[i]['candidates'])-set(test_pred_result_rank[i]))[0]
      test_q[i]['candidates'] = test_pred_result_rank[i]
      test_q[i]['candidates'].insert((len(test_q[i]['candidates'])+1)//2, missing_element)
    elif (len(test_q[i]['candidates']) < len(test_pred_result_rank[i])):
      type_c += 1
      true_list = []
      for j in range(len(test_pred_result_rank[i])):
        if test_pred_result_rank[i][j] in test_q[i]['candidates']:
          true_list.append(test_pred_result_rank[i][j])
      test_q[i]['candidates'] = true_list
  else:
    test_q[i]['candidates'] = test_pred_result_rank[i]


In [ ]:
with open('suggestion.json', 'w') as outfile:
    json.dump(test_q, outfile)

In [ ]:
files.download('suggestion.json')

In [ ]:
author_list = []
title_list = []
content_list = []
for i in range(len(test_q)):
  author_list.append("unknown author")
  title_list.append(test_q[i]['title_id'])
  content_list.append(contents[test_data['title_id']])

test_df = pd.DataFrame(list(zip(author_list, title_list, content_list)), columns=["author", "title", "content"])

In [ ]:
title_list = []
content_list = []
for i in tqdm(range(len(test_df))):
  for j in range(0, len(test_df["content"][i]), 128):
    title_list.append(test_df['title'][i])
    content_list.append(test_df['content'][i][j:j+256])
decompose_test = pd.DataFrame(list(zip(author_list, content_list)), columns=["title", "content"])

In [ ]:
X_test = tokenizer.texts_to_sequences(decompose_test['content'].values)
X_test = pad_sequences(X_test, maxlen=256)

In [ ]:
model.predict(X_test)